In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...
...,...,...,...,...
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...


In [4]:
df.shape

(57650, 4)

In [5]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [25]:
df=df.sample(5000).drop('link', axis=1).reset_index(drop = True)

In [26]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [27]:
import nltk
from nltk.stem.porter import PorterStemmer

In [28]:
stemmer = PorterStemmer()

In [29]:
def token(txt):
    token = nltk.word_tokenize(txt)
    a = [stemmer.stem(w) for w in token]
    return " ".join(a)

In [30]:
df['text'].apply(lambda x: token(x))

0       i realiz i miss a day but i 'm too wreck to ca...
1       i capricci tuoi son cose ridicol tu distrugger...
2       we were danc danc in the dark sweet romanc som...
3       my strang desir is wait here to be my light my...
4       i look into the mirror , see myself , i 'm ove...
                              ...                        
4995    see the bright new moon is rise , abov the lan...
4996    thi is my confession pen and paper i 'm gon na...
4997    lord it 's the same old tune fiddl and guitar ...
4998    what up ? i got nosk in here with me thi ai n'...
4999    you are the onli one you save me from myself m...
Name: text, Length: 5000, dtype: object

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [32]:
tfid = TfidfVectorizer(analyzer='word', stop_words='english')

In [33]:
matrix = tfid.fit_transform(df['text'])

In [34]:
df.shape

(5000, 3)

In [35]:
matrix.shape

(5000, 23400)

In [36]:
similar = cosine_similarity(matrix)

In [37]:
similar[0]

array([1.        , 0.        , 0.00475395, ..., 0.03119803, 0.00547514,
       0.02101967])

 Recommender Function

In [38]:
def recommender(song_name):
    idx = df[df['song']==song_name].index[0]
    distance = sorted(list(enumerate(similar[idx])),reverse=True,key = lambda x:x[1])
    song = []
    for s_id in distance[1:5]:
        song.append(df.iloc[s_id[0]].song)
    return song 
        

In [40]:
df.head()

,artist,song,text
0,Scorpions,Blackout,i realize i missed a day \r but i'm too wreck...
1,Neil Sedaka,I Tuoi Capricci,i capricci tuoi son cose ridicole \r tu distr...
2,Kim Wilde,Dancing In The Dark,we were dancing \r dancing in the dark \r sw...
3,INXS,Strange Desire,my strange desire \r is waiting here \r to b...
4,Enigma,Beyond,"i look into the mirror, see myself, i'm over m..."


In [41]:
recommender("Strange Desire")

['Strange', 'Desire', 'Fade Into You', 'Two Hands, One Heart']

In [42]:
import pickle

In [44]:
pickle.dump(similar, open("similarity","wb"))

In [45]:
pickle.dump(df, open("df","wb"))